<a href="https://colab.research.google.com/github/Inc139/--24--2-2/blob/main/%D0%9B%D0%B0%D0%B1_2_%D0%A8%D0%BD%D0%B5%D0%B9%D0%B4%D0%B5%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI()

# Временное хранилище заметок (список в памяти)
notes_db = []
current_id = 1

# Модель данных для заметки (PyDantic)
class Note(BaseModel):
    title: str
    content: str

class NoteResponse(Note):
    id: int

# Обработка ошибок
@app.exception_handler(HTTPException)
async def http_exception_handler(request, exc):
    return JSONResponse(
        status_code=exc.status_code,
        content={"detail": exc.detail},
    )

# Маршруты
@app.get("/notes", response_model=List[NoteResponse])
def get_all_notes():
    return notes_db

@app.get("/notes/{note_id}", response_model=NoteResponse)
def get_note(note_id: int):
    for note in notes_db:
        if note["id"] == note_id:
            return note
    raise HTTPException(status_code=404, detail="Заметка не найдена")

@app.get("/notes/stats")
def get_stats():
    return {"count": len(notes_db)}


@app.post("/notes", response_model=NoteResponse, status_code=201)
def create_note(note: Note):
    global current_id
    new_note = note.dict()
    new_note["id"] = current_id
    notes_db.append(new_note)
    current_id += 1
    return new_note

@app.put("/notes/{note_id}", response_model=NoteResponse)
def update_note(note_id: int, updated_note: Note):
    for idx, note in enumerate(notes_db):
        if note["id"] == note_id:
            notes_db[idx] = {**updated_note.dict(), "id": note_id}
            return notes_db[idx]
    raise HTTPException(status_code=404, detail="Заметка не найдена")

@app.delete("/notes/{note_id}")
def delete_note(note_id: int):
    for idx, note in enumerate(notes_db):
        if note["id"] == note_id:
            del notes_db[idx]
            return {"message": "Заметка удалена"}
    raise HTTPException(status_code=404, detail="Заметка не найдена")

@app.delete("/notes")
def delete_all_notes():
    notes_db.clear()
    return {"message": "Все заметки удалены"}